## 수정사항  : 불균형한 TARGET 데이터 generate 알고리즘 사용해서 50:50 으로 맞춰줌
## 따라서 전체 데이터 개수 10만개에서 16만개로 바뀜
## XGboost 모델 사용 : 데이터 불균형 해소로 Accuracy의 결과는 의미를 가짐
## XGboost 모델 Test Accuracy  : 97.9%


In [6]:
import pandas as pd
import numpy as np
import os

In [7]:
df = pd.read_csv('./Data_set.csv',encoding='cp949')

In [8]:
df.head()

,CUST_ID,TARGET,BNK_LNIF_CNT,CPT_LNIF_CNT,SPART_LNIF_CNT,ECT_LNIF_CNT,TOT_LNIF_AMT,TOT_CLIF_AMT,BNK_LNIF_AMT,CPT_LNIF_AMT,...,CBPT_MBSP_YN,MOBL_FATY_PRC,TEL_CNTT_QTR,NUM_DAY_SUSP,CRMM_OVDU_AMT,TLFE_UNPD_CNT,LT1Y_MXOD_AMT,PAYM_METD,LINE_STUS,MOBL_PRIN
0,1,0,1,0,0,0,9001,9001,9001,0,...,N,800000,20111,0,0,0,0,O,U,580000
1,2,0,1,0,0,0,24001,0,24001,0,...,N,500000,20143,0,0,0,0,O,U,90000
2,3,0,0,1,3,2,15001,9001,0,3001,...,Y,500000,20103,0,0,0,0,O,U,120000
3,4,1,0,2,4,2,6001,3001,0,3001,...,N,900000,20144,0,540000,0,630000,G,S,320000
4,5,0,4,0,0,0,21001,15001,21001,0,...,Y,800000,20131,0,130000,0,90000,G,U,410000


In [9]:
df.OCCP_NAME_G[:5]

0    공무원
1    자영업
2     주부
3     학생
4    공무원
Name: OCCP_NAME_G, dtype: object

In [10]:
df.TARGET.value_counts(0)

0    95946
1     4287
Name: TARGET, dtype: int64

In [17]:
nf =df.isnull().sum()
nf

CUST_ID                  0
TARGET                   0
BNK_LNIF_CNT             0
CPT_LNIF_CNT             0
SPART_LNIF_CNT           0
ECT_LNIF_CNT             0
TOT_LNIF_AMT             0
TOT_CLIF_AMT             0
BNK_LNIF_AMT             0
CPT_LNIF_AMT             0
CRDT_OCCR_MDIF           0
SPTCT_OCCR_MDIF          0
CRDT_CARD_CNT            0
CTCD_OCCR_MDIF           0
CB_GUIF_CNT              0
CB_GUIF_AMT              0
OCCP_NAME_G            464
CUST_JOB_INCM            0
HSHD_INFR_INCM           0
ACTL_FMLY_NUM            0
CUST_FMLY_NUM            0
LAST_CHLD_AGE         1027
MATE_OCCP_NAME_G     45709
MATE_JOB_INCM            0
CRDT_LOAN_CNT            0
MIN_CNTT_DATE            0
TOT_CRLN_AMT             0
TOT_REPY_AMT             0
CRLN_OVDU_RATE           0
CRLN_30OVDU_RATE         0
                     ...  
GDINS_MON_PREM           0
SVINS_MON_PREM           0
FMLY_GDINS_MNPREM        0
FMLY_SVINS_MNPREM        0
MAX_MON_PREM             0
TOT_PREM                 0
F

In [7]:
df.PAYM_METD.describe()

count     97400
unique        4
top           O
freq      56919
Name: PAYM_METD, dtype: object

In [8]:
df.PAYM_METD = df.PAYM_METD.fillna('O')

In [9]:
df.TEL_MBSP_GRAD.describe()

count     54218
unique        4
top           R
freq      19862
Name: TEL_MBSP_GRAD, dtype: object

In [10]:
df.TEL_MBSP_GRAD = df.TEL_MBSP_GRAD.fillna('R')

In [11]:
df.MATE_OCCP_NAME_G.describe() 

count     54524
unique       17
top          주부
freq      11683
Name: MATE_OCCP_NAME_G, dtype: object

In [12]:
df.MATE_OCCP_NAME_G = df.MATE_OCCP_NAME_G.fillna('주부')

In [13]:
df.LAST_CHLD_AGE.describe()

count    99206.000000
mean        12.233958
std         13.964236
min          0.000000
25%          0.000000
50%          0.000000
75%         24.000000
max         60.000000
Name: LAST_CHLD_AGE, dtype: float64

In [14]:
df.LAST_CHLD_AGE = df.LAST_CHLD_AGE.fillna(df.LAST_CHLD_AGE.mean())

In [15]:
df.OCCP_NAME_G.describe()

count     99769
unique       17
top          주부
freq      27565
Name: OCCP_NAME_G, dtype: object

In [16]:
df.OCCP_NAME_G = df.OCCP_NAME_G.fillna('주부')

In [17]:
sum(df.isnull().sum())

0

In [18]:
del df['CUST_ID']

In [19]:
df = pd.get_dummies(df)

## unbalance data dealing
### smote+enn
1. smote enn 알고리즘 사용해서 unbalance한 1의 데이터를 늘린다.(k-nn처럼 1이 군집해 있는 공간과 가까운 곳에 새로운 데이터 생성)
2. 다시 합치고 shuffle 

In [20]:
df.head()

,TARGET,BNK_LNIF_CNT,CPT_LNIF_CNT,SPART_LNIF_CNT,ECT_LNIF_CNT,TOT_LNIF_AMT,TOT_CLIF_AMT,BNK_LNIF_AMT,CPT_LNIF_AMT,CRDT_OCCR_MDIF,...,TEL_MBSP_GRAD_R,TEL_MBSP_GRAD_W,CBPT_MBSP_YN_N,CBPT_MBSP_YN_Y,PAYM_METD_G,PAYM_METD_K,PAYM_METD_O,PAYM_METD_R,LINE_STUS_S,LINE_STUS_U
0,0,1,0,0,0,9001,9001,9001,0,1,...,1,0,1,0,0,0,1,0,0,1
1,0,1,0,0,0,24001,0,24001,0,0,...,1,0,1,0,0,0,1,0,0,1
2,0,0,1,3,2,15001,9001,0,3001,1,...,1,0,0,1,0,0,1,0,0,1
3,1,0,2,4,2,6001,3001,0,3001,1,...,1,0,1,0,1,0,0,0,1,0
4,0,4,0,0,0,21001,15001,21001,0,1,...,0,1,0,1,1,0,0,0,0,1


In [45]:
x_data = df.drop('TARGET',axis=1)
y_data = df.TARGET

In [46]:
from imblearn.combine import SMOTEENN

import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA

In [47]:
print(__doc__)

X = x_data
y = y_data

print(np.shape(X))
print(np.shape(y))

# Apply SMOTE + ENN
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_sample(X, y)


Automatically created module for IPython interactive environment
(100233, 128)
(100233,)


In [63]:
df_1 = pd.DataFrame(X_resampled)
df_1.head()
df_2 = pd.DataFrame(y_resampled)
df_2.head()

,0
0,0
1,0
2,0
3,0
4,0


In [80]:
df_2.columns = ['TARGET']
df_2.head()
df_2.TARGET.value_counts(0)

1    90360
0    71529
Name: TARGET, dtype: int64

In [83]:
df_final = pd.concat([df_1,df_2],axis=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,TARGET
0,1.0,0.0,0.0,0.0,9001.0,9001.0,9001.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
1,1.0,0.0,0.0,0.0,24001.0,0.0,24001.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
2,0.0,1.0,3.0,2.0,15001.0,9001.0,0.0,3001.0,1.0,25.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0
3,1.0,0.0,1.0,1.0,141001.0,27001.0,111001.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
4,0.0,1.0,2.0,1.0,12001.0,3001.0,0.0,9001.0,121.0,121.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0


In [87]:
df_final = df_final.sample(frac=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,TARGET
10437,0.000000,1.000000,3.000000,2.000000,27001.000000,27001.000000,0.000000,3001.000000,13.000000,121.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0
30810,0.000000,0.000000,2.000000,2.000000,3001.000000,3001.000000,0.000000,0.000000,13.000000,121.0,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0
134618,1.000000,1.661577,3.441051,1.779474,26339.423214,25677.846429,3662.576786,1985.730357,29.061079,49.0,...,0.220526,0.000000,1.000000,0.779474,0.220526,0.0,0.0,0.0,1.0,1
12988,1.000000,0.000000,0.000000,0.000000,9001.000000,9001.000000,9001.000000,0.000000,1.000000,0.0,...,1.000000,0.000000,1.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0
147554,0.106898,0.106898,2.000000,1.893102,24394.361942,23111.581780,3848.447384,962.192020,1.000000,25.0,...,0.000000,0.893102,0.106898,0.000000,0.000000,1.0,0.0,0.0,1.0,1


In [88]:
data_split = int(0.7*len(df_final))
train_data = df_final[:data_split]
test_data = df_final[data_split:]

In [90]:
train_y = train_data.TARGET
train_x = train_data.drop('TARGET',axis=1)

In [91]:
test_y = test_data.TARGET
test_x = test_data.drop('TARGET',axis=1)

## Min-Max Scaling

In [92]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,5))
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.fit_transform(test_x)

In [93]:
from sklearn import linear_model
reg = linear_model.LogisticRegression()
reg.fit(train_x,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [102]:
score = reg.score(test_x,test_y)
print('Logistic Regression Accuracy : ',score)

Logistic Regression Accuracy :  0.863322832376


### 위의  ACCURACY 는 이제 의미있음 . 
#### 왜냐면 TARGET 값이 1인 데이터를 SMOTE 알고리즘으로 생성해 주어서 데이터의 균형을 맞췄기 때문에

In [96]:
from sklearn.metrics import classification_report
print(classification_report(test_y,reg.predict(test_x)))

             precision    recall  f1-score   support

          0       0.86      0.83      0.84     21439
          1       0.87      0.89      0.88     27128

avg / total       0.86      0.86      0.86     48567



In [97]:
reg2 = linear_model.LogisticRegression(class_weight='balanced')
reg2.fit(train_x,train_y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [103]:
score2 = reg2.score(test_x,test_y)
print('Logistic Regression used class weight ' ,score2)
print(classification_report(test_y,reg2.predict(test_x)))

Logistic Regression used class weight  0.859740152779
             precision    recall  f1-score   support

          0       0.84      0.85      0.84     21439
          1       0.88      0.87      0.87     27128

avg / total       0.86      0.86      0.86     48567



### 위의 결과를 보면 Balance를 맞추어 주었기 때문에 class_weight 를 걸어도 비슷한 결과가 나옴

In [99]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,normalize
from sklearn import preprocessing

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### XGboost 모델 사용 파라메터 최적화 더 필요함.

In [100]:
# fit model no training data
model = XGBClassifier(learning_rate=0.2,max_depth=7,base_score=0.5,max_delta_step=0,n_estimators=100)

pipe = Pipeline([['xg',MinMaxScaler(feature_range=(0,5))],['model',model]])
pipe.fit(train_x,train_y)

Pipeline(memory=None,
     steps=[('xg', MinMaxScaler(copy=True, feature_range=(0, 5))), ['model', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.2, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)]])

In [101]:
print(classification_report(test_y,pipe.predict(test_x)))
print("ACCURACY:",pipe.score(test_x,test_y))

             precision    recall  f1-score   support

          0       0.96      0.99      0.98     21439
          1       0.99      0.97      0.98     27128

avg / total       0.98      0.98      0.98     48567

ACCURACY: 0.979265756584


## 이제 feature 엔지니어링으로 ACCURACY 를 더 올리면됨.